#Create a Question Component - creates JAVA problems for practicing and evaluates and provides feedback for the same.

In [0]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms.cohere import Cohere
from langchain.vectorstores import FAISS
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import FAISS
import os
import numpy as np
from langchain.document_loaders import DirectoryLoader

In [0]:
# Initialize Cohere LLM
cohere_llm = Cohere(cohere_api_key="9rZaWYoS4JsQMcOMZjozd66X0DqbStcRVe8LimVQ", model="command-r-plus")

In [0]:
# Set up the retrieval function (simulate retrieving related resources)
def retrieve_relevant_info(query):
    """
    Simulate retrieval of related information (e.g., previous problems, code samples, or explanations).
    """
    # Example data: this can be replaced with actual database queries or document retrievals
    relevant_info = {
        "Inheritance and Polymorphism": "In object-oriented programming, inheritance allows one class to inherit methods and properties from another class. Polymorphism allows objects of different classes to be treated as objects of a common superclass, typically using method overriding or interfaces.",
        "common mistakes": "Common mistakes in polymorphism include not properly overriding methods, missing or incorrect constructor calls, and failure to handle edge cases such as null references."
    }

    # Return relevant information based on the query
    return relevant_info.get(query, "No relevant information found.")

In [0]:

# Problem Generator Module (RAG-Enhanced)
def generate_problem(topic):
    """
    Generates a detailed Java programming problem in the specified OOP topic, enriched with relevant context.
    """
    # Retrieve related information
    relevant_info = retrieve_relevant_info(topic)

    # Create the problem prompt enriched with retrieved info
    problem_prompt = PromptTemplate(
        input_variables=["topic", "relevant_info"],
        template=(
            "You are a Java programming instructor. Create a detailed programming problem in the area of {topic}. "
            "Use the following information to help design the problem:\n{relevant_info}\n"
            "Include a clear rubric, deliverables, and requirements. Make the problem challenging yet solvable for students."
        )
    )

    # Set up the problem generation chain
    problem_chain = LLMChain(llm=cohere_llm, prompt=problem_prompt)
    return problem_chain.run(topic=topic, relevant_info=relevant_info)

In [0]:

# Code Evaluator Module (RAG-Enhanced)
def evaluate_code(problem_statement, rubric, student_code):
    """
    Evaluates the student's Java code against the problem statement and rubric, enriched with additional context.
    """
    # Retrieve common mistakes related to the topic for context
    common_mistakes = retrieve_relevant_info("common mistakes")

    # Create the evaluation prompt enriched with retrieved info
    evaluation_prompt = PromptTemplate(
        input_variables=["problem_statement", "rubric", "student_code", "common_mistakes"],
        template=(
            "You are a Java programming evaluator. The problem statement is as follows:\n{problem_statement}\n\n"
            "Rubric for evaluation:\n{rubric}\n\n"
            "The student's code submission is:\n{student_code}\n\n"
            "Common mistakes for this topic include:\n{common_mistakes}\n\n"
            "Provide a detailed evaluation based on the rubric, analyzing the correctness, design, and any improvements needed."
        )
    )

    # Set up the evaluation chain
    evaluation_chain = LLMChain(llm=cohere_llm, prompt=evaluation_prompt)
    return evaluation_chain.run(problem_statement=problem_statement, rubric=rubric, student_code=student_code, common_mistakes=common_mistakes)

In [0]:

# Feedback Generator Module (RAG-Enhanced)
def generate_feedback(evaluation):
    """
    Generates constructive feedback for the student based on the evaluation, incorporating relevant context.
    """
    # Retrieve relevant feedback context (this could be common improvement tips or best practices)
    relevant_feedback_tips = retrieve_relevant_info("common mistakes")

    # Create the feedback prompt enriched with retrieved tips
    feedback_prompt = PromptTemplate(
        input_variables=["evaluation", "relevant_feedback_tips"],
        template=(
            "You are a Java programming instructor. Based on the following evaluation, generate constructive feedback "
            "for the student. Focus on strengths, areas of improvement, and next steps for learning:\n{evaluation}\n"
            "Use the following tips to help guide the student's learning:\n{relevant_feedback_tips}"
        )
    )

    # Set up the feedback generation chain
    feedback_chain = LLMChain(llm=cohere_llm, prompt=feedback_prompt)
    return feedback_chain.run(evaluation=evaluation, relevant_feedback_tips=relevant_feedback_tips)

In [10]:

# Example Workflow
def main():
    # Step 1: Generate a problem
    topic = "Inheritance and Polymorphism"
    problem = generate_problem(topic)
    print("Generated Problem:\n", problem)

    # Step 2: Rubric and student code (for testing purposes, these will be placeholders)
    rubric = "1. Correct use of inheritance (30 points).\n2. Implementation of polymorphic behavior (30 points).\n3. Code readability and comments (20 points).\n4. Test cases and correctness (20 points)."
    student_code = """
    class Animal {
        void sound() {
            System.out.println("Animal makes a sound");
        }
    }

    class Dog extends Animal {
        @Override
        void sound() {
            System.out.println("Dog barks");
        }
    }

    public class Main {
        public static void main(String[] args) {
            Animal obj = new Dog();
            obj.sound();
        }
    }
    """

    # Step 3: Evaluate the student's code
    evaluation = evaluate_code(problem, rubric, student_code)
    print("Evaluation:\n", evaluation)

    # Step 4: Generate feedback
    feedback = generate_feedback(evaluation)
    print("Feedback:\n", feedback)

if __name__ == "__main__":
    main()


Generated Problem:
 Here is a detailed programming problem along with a rubric, deliverables, and requirements for a Java class focusing on the concepts of Inheritance and Polymorphism:
## Problem: Zoo Management System

**Background:** 
You are developing a software system to manage animals and employees in a zoo. The zoo has a wide variety of animals, and you need to create a flexible and extensible system to handle their data and interactions. 

**Requirements:**
Develop a Java program that utilizes inheritance and polymorphism to model the zoo management system. Your program should include the following:

1. **Animals and Employees:**
   - Create a superclass `LivingBeing` that stores common attributes like name, age, and health status. This superclass should also have methods like `eat()`, `move()`, and `getDescription()`.
   - Define subclasses for specific types of animals (e.g., `Mammal`, `Bird`, `Reptile`) and employees (e.g., `Zookeeper`, `Vet`) that inherit from `LivingBeing